In [14]:
import pandas as pd 
import numpy as np 

In [15]:
heat_df = pd.read_csv('./industry_heat_demand_characterization_nrel.csv', index_col=0, encoding='windows-1252',header=0)

## Explore and select data (high temperature)

In [16]:
print(heat_df.columns)

Index(['CITY', 'COUNTY', 'COUNTY_FIPS', 'Coal', 'Diesel', 'END_USE',
       'FACILITY_ID', 'FINAL_NAICS_CODE', 'FUEL_TYPE', 'FUEL_TYPE_BLEND',
       'FUEL_TYPE_OTHER', 'LPG_NGL', 'MECS_NAICS', 'Natural_gas', 'Other',
       'Process_byp', 'Pulp_Paper', 'REPORTING_YEAR', 'Residual_fuel_oil',
       'STATE', 'Temp_degC', 'Total', 'UNIT_NAME', 'UNIT_TYPE', 'for_EU_sum',
       'Temp_Band', 'Biogenic', 'MMTCO2E'],
      dtype='object')


In [17]:
# Drop null demand and convert total from TJ to MJ 
heat_df.drop(heat_df.index[(heat_df["Total"] ==0)],axis=0,inplace=True)
heat_df['Heat demand (MJ/year)'] = heat_df['Total']*1e6
heat_df.drop(columns=['Total'], inplace=True)

In [18]:
for col in heat_df.columns: 
  print('{} takes the following values: {}'.format(col, heat_df[col].unique()))

CITY takes the following values: ['SPARROWS POINT' 'Ashland' 'MIDDLETOWN' 'DEARBORN' 'ROANOKE' 'ECORSE'
 'Braddock' 'GARY' 'CLEVELAND' 'TRINITY' 'IRVINE' 'NEW CASTLE' 'FAIRFIELD'
 'WARREN' 'East Chicago' 'BURNS HARBOR' 'EAST CHICAGO' 'GRANITE CITY'
 'ASHLAND' 'READING' 'VANDERGRIFT' 'PEORIA' 'BURNHAM' 'CRAWFORDSVILLE'
 'ROCKPORT' 'CARTERSVILLE' 'SEATTLE' 'CANTON' 'WASHINGTON' 'CLAYMONT'
 'Lorain' 'CHARLOTTE' 'LONGVIEW' 'SAYREVILLE' 'AMBRIDGE' 'CALVERT'
 'Columbus' 'HUNTINGTON' 'LEECHBURG' 'MONROE' 'JACKSON' 'LAPLACE'
 'Youngstown' 'PUEBLO' 'Fairless Hills' 'BLYTHEVILLE' 'CUYAHOGA HEIGHTS'
 'Jackson' 'COFIELD' 'MUSCATINE' 'BALDWIN' 'ALTON' 'VINTON' 'COATESVILLE'
 'CONSHOHOCKEN' 'MEMPHIS' 'AUBURN' 'BIRMINGHAM' 'NORFOLK' 'AXIS'
 'West Mifflin' 'PLYMOUTH' 'STEELTON' 'Tuscaloosa' 'HUGER' 'MANSFIELD'
 'PETERSBURG' 'BRIDGEVILLE' 'GALLATIN' 'MIDLAND' 'BRACKENRIDGE' 'PORTAGE'
 'KOPPEL' 'BUTLER' 'JEWETT' 'MIDLOTHIAN' 'MARION' 'KNOXVILLE' 'ROSE CITY'
 'Ghent' 'RANCHO CUCAMONGA' 'Portland' 'MCMINN

In [19]:
# We are interested in displacing natural gas for temperatures higher than outlet temperatures of ANRs
# Max delivered temperature is : thermal transfer efficiency x max outlet temperature = TTE(HTGR) x Outlet temp (HTGR) = 
# 90% x 950 deg C = 855 deg C
CUTOFF = 855
print('All entries {}'.format(len(heat_df)))
hot_df = heat_df[heat_df['Temp_degC']>=CUTOFF]
print('Only above cutoff ({} deg C): {}, {} % of the data'.format(CUTOFF, len(hot_df), np.round(100*len(hot_df)/len(heat_df),2) ))

All entries 81847
Only above cutoff (855 deg C): 6521, 7.97 % of the data


In [20]:
for col in hot_df.columns: 
  print('{} takes the following values: {}'.format(col, hot_df[col].unique()))

CITY takes the following values: ['SPARROWS POINT' 'Ashland' 'MIDDLETOWN' 'DEARBORN' 'ROANOKE' 'ECORSE'
 'Braddock' 'GARY' 'CLEVELAND' 'TRINITY' 'IRVINE' 'NEW CASTLE' 'FAIRFIELD'
 'WARREN' 'East Chicago' 'BURNS HARBOR' 'EAST CHICAGO' 'GRANITE CITY'
 'ASHLAND' 'READING' 'VANDERGRIFT' 'PEORIA' 'BURNHAM' 'CRAWFORDSVILLE'
 'ROCKPORT' 'CARTERSVILLE' 'SEATTLE' 'CANTON' 'WASHINGTON' 'CLAYMONT'
 'Lorain' 'CHARLOTTE' 'LONGVIEW' 'SAYREVILLE' 'AMBRIDGE' 'CALVERT'
 'Columbus' 'HUNTINGTON' 'LEECHBURG' 'MONROE' 'JACKSON' 'LAPLACE'
 'Youngstown' 'PUEBLO' 'Fairless Hills' 'BLYTHEVILLE' 'CUYAHOGA HEIGHTS'
 'Jackson' 'COFIELD' 'MUSCATINE' 'BALDWIN' 'ALTON' 'VINTON' 'COATESVILLE'
 'CONSHOHOCKEN' 'MEMPHIS' 'AUBURN' 'BIRMINGHAM' 'NORFOLK' 'AXIS'
 'West Mifflin' 'PLYMOUTH' 'STEELTON' 'Tuscaloosa' 'HUGER' 'MANSFIELD'
 'PETERSBURG' 'BRIDGEVILLE' 'GALLATIN' 'MIDLAND' 'BRACKENRIDGE' 'PORTAGE'
 'KOPPEL' 'BUTLER' 'JEWETT' 'MIDLOTHIAN' 'MARION' 'KNOXVILLE' 'ROSE CITY'
 'Ghent' 'RANCHO CUCAMONGA' 'Portland' 'MCMINN

We need to remove entries with a type of fuel corresponding to a by-product of the process itself

In [21]:
NON_AUTHORIZED_FUEL_TYPE = ['Blast Furnace Gas', 'Coke Oven Gas', 'Fuel Gas', '0', 'Used Oil']
NON_AUTHORIZED_FUEL_TYPE_BLEND = ['Process Off-gas', 'Scrap Fat (from W-4 Tank)', 'Nitrile Pitch (from W-3 Tank)','Biogenic Process Derived Fuel (Glidfuel)',
                            'Biogenic Process Derived Fuel', 'Sweet Gas  Return', 'Biogeninc Process Derived Fuel (PDF)',
                             'Biogenic Process Derived Fuel (PDF)','FurnGas', 'Fuel Gas']
ng_df = hot_df[hot_df['FUEL_TYPE'].isin(['Natural Gas (Weighted U.S. Average)', 'Mixed (Industrial sector)'])]
ng_df = ng_df[~(ng_df['FUEL_TYPE_BLEND'].isin(NON_AUTHORIZED_FUEL_TYPE_BLEND))]
print(len(ng_df))
print(np.round(100*len(ng_df)/len(heat_df),2), '% of total entries')
print(np.round(100*len(ng_df)/len(hot_df),2), '% of hot entries')

3545
4.33 % of total entries
54.36 % of hot entries


## Convert remaining data in equivalent hydrogen demand

In [22]:
ng_df = ng_df[['CITY', 'COUNTY', 'FACILITY_ID', 'FUEL_TYPE', 'REPORTING_YEAR', 'STATE', 'Temp_degC',  'Natural_gas', 'Other',
               'UNIT_NAME', 'UNIT_TYPE', 'MMTCO2E', 'Heat demand (MJ/year)']]
ng_df


,CITY,COUNTY,FACILITY_ID,FUEL_TYPE,REPORTING_YEAR,STATE,Temp_degC,Natural_gas,Other,UNIT_NAME,UNIT_TYPE,MMTCO2E,Heat demand (MJ/year)
0,SPARROWS POINT,Baltimore,1000553,Natural Gas (Weighted U.S. Average),2010,MD,1100.0,328.280712,0.0,Pennwood Boiler #1,"OB (Boiler, other)",0.016527,3.282807e+08
1,SPARROWS POINT,Baltimore,1000553,Natural Gas (Weighted U.S. Average),2010,MD,1100.0,666.678118,0.0,Pennwood Boiler #2,"OB (Boiler, other)",0.033563,6.666781e+08
2,SPARROWS POINT,Baltimore,1000553,Natural Gas (Weighted U.S. Average),2010,MD,1100.0,172.814781,0.0,Pennwood Boiler #3,"OB (Boiler, other)",0.008700,1.728148e+08
3,SPARROWS POINT,Baltimore,1000553,Natural Gas (Weighted U.S. Average),2010,MD,1100.0,2925.987834,0.0,CP-NG Combustion Sources,OCS (Other combustion source),0.147303,2.925988e+09
4,SPARROWS POINT,Baltimore,1000553,Natural Gas (Weighted U.S. Average),2010,MD,1100.0,318.073057,0.0,Pennwood Boiler #4,"OB (Boiler, other)",0.016013,3.180731e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66917,STERLINGTON,OUACHITA,1002263,Natural Gas (Weighted U.S. Average),2015,LA,900.0,188.188061,0.0,UR-01,PRH (Process Heater),0.009474,1.881881e+08
66918,PACE,SANTA ROSA COUNTY,1008023,Natural Gas (Weighted U.S. Average),2015,FL,900.0,27.321438,0.0,#1 Amines Gas Fired Heater,PRH (Process Heater),0.001375,2.732144e+07
66919,PACE,SANTA ROSA COUNTY,1008023,Natural Gas (Weighted U.S. Average),2015,FL,900.0,31.675848,0.0,#4 Amines Gas Fired Heater,PRH (Process Heater),0.001595,3.167585e+07
66920,PACE,SANTA ROSA COUNTY,1008023,Natural Gas (Weighted U.S. Average),2015,FL,900.0,13.685573,0.0,#2 Amines Gas Fired Heater,PRH (Process Heater),0.000689,1.368557e+07


In [23]:
# Load the conversion spreadsheet for hhv of hydrogen
hhv_map = pd.read_excel('fuels_HHV_industry_heat.xlsx', sheet_name='hhv', skiprows=1, index_col='fuel')
hhv_hydrogen = hhv_map.at['Hydrogen','HHV (MJ/kg)']
print(hhv_hydrogen)

141.88


In [24]:
# Convert heat demand to mass of hydrogen demand
ng_df['H2 demand (kg/year)'] = ng_df['Heat demand (MJ/year)']/hhv_hydrogen


### Yearly basis demand, average and max demands

In [25]:
# The data includes 6 reporting years: 2010 to 2015. We include the min, avg and max of heat demand in the pre-processed demand spreadsheet
stats_df = ng_df[['FACILITY_ID', 'Natural_gas', 'Other', 'MMTCO2E', 'Heat demand (MJ/year)', 'H2 demand (kg/year)']]
max_df = stats_df.groupby('FACILITY_ID').max()
avg_df = stats_df.groupby('FACILITY_ID').mean()
min_df = stats_df.groupby('FACILITY_ID').min()


In [26]:
# Save everything
with pd.ExcelWriter('../h2_demand_industry_heat.xlsx') as writer: 
  ng_df.to_excel(writer, index=False, sheet_name='all_years')
  max_df.to_excel(writer, sheet_name='max')
  avg_df.to_excel(writer, sheet_name='mean')
  min_df.to_excel(writer, sheet_name='min')